In [3]:
# main.py - Iron Lady Chatbot with Hugging Face Intent Classification
import gradio as gr
from transformers import pipeline
from groq import Groq
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import warnings
warnings.filterwarnings("ignore")

GROQ_API_KEY = "GROQ_API"
TEXT_FILE_PATH = "iron_lady_data.txt"

class SimpleIntentClassifier:
    def __init__(self):
        # Load a pre-trained zero-shot classification model
        try:
            self.classifier = pipeline(
                "zero-shot-classification",
                model="facebook/bart-large-mnli",
                device=0
            )
        except Exception as e:
            print(f"Could not load BART model, using simpler model: {e}")
            # Fallback to a smaller model
            self.classifier = pipeline(
                "zero-shot-classification", 
                model="microsoft/DialoGPT-medium"
            )
        
        # Define our intent labels
        self.intent_labels = [
            "greeting",
            "goodbye", 
            "ask about programs",
            "ask about mentors",
            "ask about cost and pricing",
            "ask about results and success stories",
            "ask about duration and timeline",
            "general question"
        ]
    
    def classify_intent(self, text):
        """Classify user intent using zero-shot classification"""
        try:
            result = self.classifier(text, self.intent_labels)
            
            # Get the best prediction
            best_intent = result['labels'][0]
            confidence = result['scores'][0]
            
            # Convert to simple intent names
            intent_mapping = {
                "greeting": "greet",
                "goodbye": "goodbye",
                "ask about programs": "ask_programs",
                "ask about mentors": "ask_mentors", 
                "ask about cost and pricing": "ask_cost",
                "ask about results and success stories": "ask_results",
                "ask about duration and timeline": "ask_duration",
                "general question": "general"
            }
            
            intent = intent_mapping.get(best_intent, "general")
            
            # Only use the classification if confidence is reasonable
            if confidence > 0.3:
                return intent
            else:
                return "general"
                
        except Exception as e:
            print(f"Intent classification error: {e}")
            return "general"

def build_rag():
    """Build RAG system for Iron Lady content"""
    loader = TextLoader(TEXT_FILE_PATH, encoding='utf-8')
    data = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(data)
    
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    
    llm = ChatGroq(api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
    )
    
    return qa_chain

def get_system_prompt_for_intent(intent):
    """Get specialized system prompt based on detected intent"""
    prompts = {
        "ask_programs": (
            "You are an enthusiastic Iron Lady program consultant. "
            "Focus on the three flagship programs: Leadership Essentials (executive presence), "
            "100 Board Members (corporate board preparation), and C-Suite League (business warfare tactics). "
            "Highlight unique benefits and what makes each program special."
        ),
        
        "ask_mentors": (
            "You are showcasing Iron Lady's world-class mentorship team. "
            "Emphasize the impressive credentials: Rajesh Bhat (Founder, TEDx speaker), "
            "Suvarna Hegde (CEO, Business War Tactics creator), Simon Newman (Ex-CEO Aviva), "
            "and Sridhar Sambandam (Ex-CEO Bajaj Auto). Focus on their expertise and guidance quality."
        ),
        
        "ask_cost": (
            "You are a helpful investment advisor discussing Iron Lady's value proposition. "
            "Emphasize the ROI - many graduates earn 1 crore+ annually. Position it as a strategic "
            "career investment. If exact pricing isn't available, focus on value and suggest contacting for details."
        ),
        
        "ask_results": (
            "You are highlighting Iron Lady's impressive track record and transformations. "
            "Emphasize concrete numbers: 78,000+ women trained, 117+ members earning 1 crore+ annually. "
            "Make the success stories tangible and inspiring. Show how careers and incomes have transformed."
        ),
        
        "ask_duration": (
            "You are explaining Iron Lady's comprehensive learning approach and timelines. "
            "Focus on the blended delivery model: in-person sessions, app-based learning, and personal mentoring. "
            "Emphasize how the duration ensures thorough skill development and lasting transformation."
        ),
        
        "greet": (
            "You are a warm, welcoming Iron Lady representative greeting a potential participant. "
            "Be enthusiastic about helping them explore leadership development opportunities. "
            "Ask how you can help them with their leadership journey."
        ),
        
        "goodbye": (
            "You are thanking someone for their interest in Iron Lady. "
            "Be encouraging about their leadership potential and invite them to reach out anytime. "
            "Leave them feeling positive about their leadership journey."
        )
    }
    
    return prompts.get(intent, 
        "You are a knowledgeable Iron Lady assistant. Provide clear, concise, supportive information "
        "about leadership development programs for women. Be conversational and helpful."
    )

def generate_response(text, intent_classifier, qa_chain):
    """Generate response using HuggingFace intent + RAG + Groq"""
    
    # Get intent classification
    intent = intent_classifier.classify_intent(text)
    print(f"🎯 Detected intent: {intent}")  # Debug info
    
    # Handle simple greetings/goodbyes without RAG
    if intent in ["greet", "goodbye"]:
        client = Groq(api_key=GROQ_API_KEY)
        chat_completion = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": get_system_prompt_for_intent(intent)},
                {"role": "user", "content": text}
            ]
        )
        return chat_completion.choices[0].message.content
    
    # Use RAG for information-seeking questions
    try:
        result = qa_chain.invoke({"query": text})
        rag_answer = result["result"]
        
        # Generate tailored response based on intent
        client = Groq(api_key=GROQ_API_KEY)
        chat_completion = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": get_system_prompt_for_intent(intent)},
                {"role": "user", "content": f"User asked: {text}\n\nContext from Iron Lady data: {rag_answer}"}
            ]
        )
        return chat_completion.choices[0].message.content
        
    except Exception as e:
        print(f"Error generating response: {e}")
        return "I apologize, but I encountered an issue. Please try rephrasing your question about Iron Lady's leadership programs."

# Initialize systems
print("🚀 Loading models...")
intent_classifier = SimpleIntentClassifier()
qa_chain = build_rag()
print("✅ All systems ready!")

def update_chat(user_input, history):
    if not user_input.strip():
        return history, ""
    
    response = generate_response(user_input, intent_classifier, qa_chain)
    history.append((user_input, response))
    
    return history, ""

def clear_chat():
    return []

# Gradio Interface
with gr.Blocks(theme="default", title="Iron Lady AI Assistant") as demo:
    
    chatbot = gr.Chatbot(
        bubble_full_width=False, 
        height=600,
    )
    
    with gr.Row():
        user_msg = gr.Textbox(
            show_label=False, 
            placeholder="Type your question here...",
            scale=4
        )
        send_btn = gr.Button("Send ", variant="primary", scale=1)
    
    with gr.Row():
        clear_btn = gr.Button("Clear Chat 🗑️", variant="secondary")
    
    # Event handlers
    send_btn.click(fn=update_chat, inputs=[user_msg, chatbot], outputs=[chatbot, user_msg])
    user_msg.submit(fn=update_chat, inputs=[user_msg, chatbot], outputs=[chatbot, user_msg])
    clear_btn.click(fn=clear_chat, outputs=[chatbot])

if __name__ == "__main__":
    demo.launch(share=True, inbrowser=True)

🚀 Loading models...
✅ All systems ready!
* Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


🎯 Detected intent: ask_programs
🎯 Detected intent: ask_programs
🎯 Detected intent: ask_programs
🎯 Detected intent: general
🎯 Detected intent: ask_mentors
